#  Clickbait Detection: 최종 확정 모델 (Model C)
## L2 제거 + Dropout 균형 (0.2) - 발표용 최종 버전

---

### ✅ Model C 사양
- **구조**: Siamese Network + MLP (512 → 256 → 128 → 64)
- **Regularization**:
  - ❌ L2 제거 (과도한 규제 해제)
  - ✅ Dropout 0.2로 균일화 (체계적 규제)
  - ✅ BatchNormalization 유지
- **Optimizer**: Adam (5e-4)
- **Epochs**: 30 (검증된 설정)
- **예상 성능**: Val AUC 0.7767 (A/B 테스트 검증)

### 📊 A/B 테스트 결과
| Model | Val AUC | Gap | 선택 |
|-------|---------|-----|------|
| A (원본) | 0.7706 | 0.001 | ❌ 과소적합 |
| B (L2 제거) | 0.7769 | 0.028 | ✅ 1등 |
| **C (선택)** | **0.7767** | **0.028** | **✅ 최종 선택** |
| D (Higher LR) | 0.7753 | 0.022 | - |
| E (Min Reg) | 0.7756 | 0.043 | - |

**선택 이유**: B와 성능 동등 + 더 체계적인 설명 + 포트폴리오 가치 높음

## 1️⃣ 환경 설정

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Multiply, Lambda, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
from huggingface_hub import hf_hub_download
import pickle
from datetime import datetime

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Seed 고정 (재현성)
def set_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(1234)
print(" Seed 설정 완료 (1234)")

✅ Seed 설정 완료 (1234)


## 2️⃣ 데이터 로드

In [ ]:
print(" 데이터 로드 중...\n")

train_path = hf_hub_download("jjanoong2/clickbait_embeddings", "train_embeddings.npz", repo_type="dataset")
val_path = hf_hub_download("jjanoong2/clickbait_embeddings", "val_embeddings.npz", repo_type="dataset")

train = np.load(train_path)
val = np.load(val_path)

X_title_train = train['title_embeddings'].astype('float32')
X_content_train = train['content_embeddings'].astype('float32')
y_train = train['labels'].astype('float32')

X_title_val = val['title_embeddings'].astype('float32')
X_content_val = val['content_embeddings'].astype('float32')
y_val = val['labels'].astype('float32')

print(" 데이터 로드 완료!")
print(f"   Train: {len(X_title_train):,}개")
print(f"   Val:   {len(X_title_val):,}개")
print(f"   Feature Dim: {X_title_train.shape[1]}차원 (BERT Embedding)")

📥 데이터 로드 중...



train_embeddings.npz:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

val_embeddings.npz:   0%|          | 0.00/424M [00:00<?, ?B/s]

✅ 데이터 로드 완료!
   Train: 466,344개
   Val:   116,588개
   Feature Dim: 768차원 (BERT Embedding)


## 3️⃣ Model C 정의 (최종 확정 버전)

In [ ]:
def create_model_c(name='Model_C_Final', lr=5e-4):
    """
    Model C: L2 제거 + Dropout 균형 (0.2)

    핵심 개선사항:
    1. L2 Regularization 완전 제거 (과도한 규제 해제)
    2. Dropout을 0.2로 균일화 (체계적 규제 전략)
    3. 512 → 256 → 128 → 64 구조 유지 (검증됨)
    """
    set_seed(1234)

    # Inputs
    title_input = Input(shape=(768,), name='title_input')
    content_input = Input(shape=(768,), name='content_input')

    # Interaction Features (Siamese Core)
    multiply = Multiply()([title_input, content_input])
    diff = Lambda(lambda x: tf.abs(x[0] - x[1]))([title_input, content_input])
    merged = Concatenate()([title_input, content_input, multiply, diff])

    # MLP Layers with Balanced Dropout (0.2)
    # Layer 1: 3072 → 512
    h = Dense(512, activation='relu', kernel_initializer='he_normal', name='dense_1')(merged)
    h = BatchNormalization(name='bn_1')(h)
    h = Dropout(0.2, name='dropout_1')(h)  # 균일화

    # Layer 2: 512 → 256
    h = Dense(256, activation='relu', kernel_initializer='he_normal', name='dense_2')(h)
    h = BatchNormalization(name='bn_2')(h)
    h = Dropout(0.2, name='dropout_2')(h)  # 균일화

    # Layer 3: 256 → 128
    h = Dense(128, activation='relu', kernel_initializer='he_normal', name='dense_3')(h)
    h = BatchNormalization(name='bn_3')(h)
    h = Dropout(0.2, name='dropout_3')(h)  # 균일화

    # Layer 4: 128 → 64
    h = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense_4')(h)
    h = BatchNormalization(name='bn_4')(h)
    # 마지막 레이어는 Dropout 없음 (정보 보존)

    # Output
    output = Dense(1, activation='sigmoid', name='output')(h)

    model = Model(inputs=[title_input, content_input], outputs=output, name=name)

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    return model

print(" Model C 정의")

✅ Model C 정의 완료


## 4️⃣ 모델 생성 및 확인

In [ ]:
model = create_model_c()
model.summary()

# 파라미터 수 계산
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"\n📊 Total Trainable Parameters: {trainable_params:,}")

Model: "Model_C_Final"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ title_input         │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ content_input       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 768)       │          0 │ title_input[0][0… │
│                     │                   │            │ content_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 768)       │          0 │ title_input[0][0… │
│                     │                   │            │ content_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3072)      │          0 │ title_input[0][0… │
│ (Concatenate)       │                   │            │ content_input[0]… │
│                     │                   │            │ multiply[0][0],   │
│                     │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │  1,573,376 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_1                │ (None, 512)       │      2,048 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ bn_1[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_2                │ (None, 256)       │      1,024 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ bn_2[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_3                │ (None, 128)       │        512 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ bn_3[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_4                │ (None, 64)        │        256 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ bn_4[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,749,761 (6.67 MB)

 Trainable params: 1,747,841 (6.67 MB)

 Non-trainable params: 1,920 (7.50 KB)


📊 Total Trainable Parameters: 1,747,841


## 5️⃣ Callbacks 설정

In [ ]:
# Callbacks 설정 (A/B 테스트와 동일)
callbacks_list = [
    EarlyStopping(
        monitor='val_auc',
        patience=4,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        mode='max',
        verbose=1
    ),
    ModelCheckpoint(
        filepath='model_c_best.keras',
        monitor='val_auc',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

print(" Callbacks 설정 완료")
print("   - EarlyStopping (patience=4)")
print("   - ReduceLROnPlateau (patience=2)")
print("   - ModelCheckpoint (최고 성능 자동 저장)")

✅ Callbacks 설정 완료
   - EarlyStopping (patience=4)
   - ReduceLROnPlateau (patience=2)
   - ModelCheckpoint (최고 성능 자동 저장)


## 6️⃣ 학습 시작 (30 Epochs)

In [ ]:
print("="*70)
print(" Model C 학습 시작")
print("="*70)
print(f"Strategy: L2 제거 + Dropout 균형 (0.2)")
print(f"Epochs: 30")
print(f"Batch Size: 256")
print(f"예상 시간: 20~25분")
print(f"목표 Val AUC: 0.7767")
print("="*70 + "\n")

start_time = datetime.now()

history = model.fit(
    [X_title_train, X_content_train],
    y_train,
    validation_data=([X_title_val, X_content_val], y_val),
    epochs=30,
    batch_size=256,
    callbacks=callbacks_list,
    verbose=1
)

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds() / 60

print("\n" + "="*70)
print(" 학습 완료!")
print(f"   소요 시간: {training_time:.1f}분")
print("="*70)

🚀 Model C 학습 시작
Strategy: L2 제거 + Dropout 균형 (0.2)
Epochs: 30
Batch Size: 256
예상 시간: 20~25분
목표 Val AUC: 0.7767

Epoch 1/30
1822/1822 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6604 - auc: 0.7021 - loss: 0.6090
Epoch 1: val_auc improved from -inf to 0.75561, saving model to model_c_best.keras
1822/1822 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.6604 - auc: 0.7021 - loss: 0.6090 - val_accuracy: 0.7025 - val_auc: 0.7556 - val_loss: 0.5452 - learning_rate: 5.0000e-04
Epoch 2/30
1819/1822 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7054 - auc: 0.7502 - loss: 0.5405
Epoch 2: val_auc improved from 0.75561 to 0.76384, saving model to model_c_best.keras
1822/1822 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7054 - auc: 0.7502 - loss: 0.5405 - val_accuracy: 0.7096 - val_auc: 0.7638 - val_loss: 0.5341 - learning_rate: 5.0000e-04
Epoch 3/30
1818/1822 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7095 - auc: 0.7586 - loss: 0.5336
Epoch 3: val_auc improved from 0.76384 to 0.76611, s

## 7️⃣ 결과 분석

In [ ]:
# 최종 성능
best_val_auc = max(history.history['val_auc'])
best_epoch = np.argmax(history.history['val_auc']) + 1
final_train_auc = history.history['auc'][-1]
final_val_auc = history.history['val_auc'][-1]
final_gap = final_train_auc - final_val_auc

print("\n" + "="*70)
print(" Model C 최종 성능")
print("="*70)
print(f"\n Best Validation AUC: {best_val_auc:.4f} (Epoch {best_epoch})")
print(f"\n Final Metrics:")
print(f"   Train AUC:     {final_train_auc:.4f}")
print(f"   Val AUC:       {final_val_auc:.4f}")
print(f"   Gap:           {final_gap:.4f}")

# Gap 분석
if final_gap < 0:
    gap_status = " 과소적합 (Val > Train)"
elif final_gap < 0.03:
    gap_status = " 정상 (건강한 학습)"
elif final_gap < 0.10:
    gap_status = " 과적합 경향"
else:
    gap_status = "❌ 과적합"

print(f"\n Gap 분석: {gap_status}")

# 원본과 비교
baseline_auc = 0.7706
improvement = ((best_val_auc - baseline_auc) / baseline_auc) * 100

print(f"\n 개선율:")
print(f"   원본 (Model A): {baseline_auc:.4f}")
print(f"   Model C:        {best_val_auc:.4f}")
print(f"   개선:           +{improvement:.2f}%")
print("="*70)


📊 Model C 최종 성능

🏆 Best Validation AUC: 0.7782 (Epoch 21)

📈 Final Metrics:
   Train AUC:     0.7999
   Val AUC:       0.7780
   Gap:           0.0219

🔍 Gap 분석: ✅ 정상 (건강한 학습)

💯 개선율:
   원본 (Model A): 0.7706
   Model C:        0.7782
   개선:           +0.98%
